In [1]:
import pandas as pd
from backtesting import Backtest, Strategy


/opt/homebrew/Caskroom/miniconda/base/envs/bot/lib/python3.13/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

# Reading in data and setting up training and testing sets
- Here we have set up a 70 / 30 training / testing set data split
- The reason for setting up training and testing sets is so we can optimise parameters on the training set, and then test them on our testing set to see if the parameters work on unseen data

In [2]:
df = pd.read_csv('SOL_1h_data.csv', parse_dates=True, index_col=0)
print(df)

#only use data from 2024 onwards
# df = df[df.index > pd.Timestamp('12-01-2024')]

split_date = df.index[int(len(df) * 0.7)]

df_train = df.loc[:split_date]
df_test  = df.loc[split_date:]

print('Training set:')
print(df_train.describe())
print('Testing set:')
print(df_test.describe())





                           Open        High         Low       Close
timestamp                                                          
2022-02-03 14:00:00   97.051725   97.349481   96.520616   96.895000
2022-02-03 15:00:00   96.895000   99.003095   96.655393   98.338750
2022-02-03 16:00:00   98.354113   99.132500   97.366627   97.431528
2022-02-03 17:00:00   97.431250   98.085687   96.160092   97.749684
2022-02-03 18:00:00   97.750000   98.000000   96.908573   97.660099
...                         ...         ...         ...         ...
2025-11-05 15:00:00  158.696464  160.495030  157.917890  160.464120
2025-11-05 16:00:00  160.462000  162.017313  158.567519  161.816408
2025-11-05 17:00:00  161.820791  162.456907  160.913136  162.024900
2025-11-05 18:00:00  162.024651  163.882646  161.696269  163.134821
2025-11-05 19:00:00  163.134022  163.377325  162.250555  162.369764

[32768 rows x 4 columns]
Training set:
               Open          High           Low         Close
count  22938.0

# Defining our strategy and its parameters
- We define our strategy as `SMAStrategy` which is an instance of `Strategy` (a built in class)
- We define the parameters that our strategy will use
- Our strategy has to have these two functions: `init` and `next`
## `init` function
- init (initialise) is the function ran when the strategy is created
- we use this function to define the logic that our trading bot will use
- in this example we precompute short and long simple moving averages (SMAs) of the Close price
- these precomputed values can then easily be used in the next function

## `next` function
- during the backtest, the next function will be ran on every single data point
- this is where we include the actual trade execution logic
- here we will enter a position (buy) when the short SMA crosses above the long SMA
- we will close the position (sell) when the short SMA crosses below the long SMA
- NOTE: this is simple logic where we only buy and sell spot, no shorts, we always buy with maximum available capital

In [3]:
class SMAStrategy(Strategy):
    sma_short_period = 10
    sma_long_period = 30

    def init(self):
        close_prices = pd.Series(self.data.Close, index=self.data.index)
        sma_short = close_prices.rolling(window=self.sma_short_period).mean()
        sma_long = close_prices.rolling(window=self.sma_long_period).mean()
        self.sma_short = self.I(lambda: sma_short, name='SMA_SHORT')
        self.sma_long = self.I(lambda: sma_long, name='SMA_LONG')

    def next(self):

        #short term sma crosses above long term, signals reversal in momentum
        #buy when crossover first happens, when prev candle did not cross
        if self.sma_short[-1] > self.sma_long[-1] and self.sma_short[-2] <= self.sma_long[-2]:
            self.buy()
        elif self.sma_short[-1] < self.sma_long[-1] and self.sma_short[-2] >= self.sma_long[-2]:
            self.position.close()



# Running a backtest on the strategy
## Backtest function parameter explanation:
- `df_train` - our dataset that we will run the strategy on
- `SMAStrategy` - the instance of Strategy that we defined, the trading logic to be backtested
- `cash` - how much money the backtest will begin with
- `exclusive_orders` - whether or not we can have multiple trades open at the same time, in this instance True means we cannot
- `finalize_trades` - at the end of the backtest any open trades will be closed at the final price in the dataset

## Displaying key performance metrics
- here we print the stats from the backtest
- the power of backtesting.py is these metrics, they are all calculated for us allowing us to easily evaluate a strategy across these metrics

In [4]:
bt_train = Backtest(df_train, SMAStrategy, cash=1000000, exclusive_orders=True, finalize_trades=True)
stats = bt_train.run()
print(stats)

Start                     2022-02-03 14:00:00
End                       2024-09-22 05:00:00
Duration                    961 days 15:00:00
Exposure Time [%]                    50.47955
Equity Final [$]                5292485.28393
Equity Peak [$]                 6026145.93697
Return [%]                          429.24853
Buy & Hold Return [%]                33.23717
Return (Ann.) [%]                    88.17824
Volatility (Ann.) [%]               149.79954
CAGR [%]                             88.22464
Sharpe Ratio                          0.58864
Sortino Ratio                           2.333
Calmar Ratio                          1.38915
Alpha [%]                           414.62559
Beta                                  0.43996
Max. Drawdown [%]                   -63.47623
Avg. Drawdown [%]                     -5.6125
Max. Drawdown Duration      394 days 23:00:00
Avg. Drawdown Duration       10 days 04:00:00
# Trades                                  451
Win Rate [%]                      

# Plotting the strategy to see a visualisation of the performance

In [5]:
bt_train.plot()


/opt/homebrew/Caskroom/miniconda/base/envs/bot/lib/python3.13/site-packages/backtesting/_plotting.py:141: UserWarning: Data contains too many candlesticks to plot; downsampling to '4h'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "


GridPlot(id='p1308', ...)

# Now to optimise the strategy
- Our strategy has parameters (e.g. `sma_short_period`, `sma_long_period`) we can optimise the strategy to find the best parameters
- We can optimise for whatever we would like, in this instance we optimise for Sharpe Ratio
- The optimise function will simply run through all possible combinations of parameters and find the one with the highest Sharpe Ratio
- This can be slow, so we can add constraints for parameter combinations that we will allow. e.g. `sma_short_period` must be lower than `sma_long_period`


In [6]:
def optimise(series):
    return series['Sharpe Ratio']

def constraint(series):
    return series['sma_short_period'] < series['sma_long_period']


# Running the optimisation
- We provide ranges of values for each parameter that we want to be tested
- `sma_short_period=range(10,20,5)` will test all combinations of the short SMA period from 10 to 20 incrementing by 5 each time. That is 10, 15, 20
- as it is doing an exhaustive search, running backtests for all combinations, it can take a long time if we specify lots of values
- take note that we specified to maximize for our optimise function (`maximize=optimise`), so it will find the parameters that maximise for whatever our optimise function returns. 
- It is important to note that if we wanted to optimise to minimise a metric we would change this, e.g. minimise the maximum drawdown faced by the strategy


In [7]:
stats_optimised = bt_train.optimize(
    sma_short_period=range(10, 20, 5), 
    sma_long_period=range(20,50,5), 
    maximize=optimise, 
    constraint=constraint)

/opt/homebrew/Caskroom/miniconda/base/envs/bot/lib/python3.13/site-packages/backtesting/backtesting.py:1624: RuntimeWarning: If you want to use multi-process optimization with `multiprocessing.get_start_method() == 'spawn'` (e.g. on Windows),set `backtesting.Pool = multiprocessing.Pool` (or of the desired context) and hide `bt.optimize()` call behind a `if __name__ == '__main__'` guard. Currently using thread-based paralellism, which might be slightly slower for non-numpy / non-GIL-releasing code. See https://github.com/kernc/backtesting.py/issues/1256
  output = _optimize_grid()


In [8]:
print(stats_optimised)
stats_optimised._strategy

Start                     2022-02-03 14:00:00
End                       2024-09-22 05:00:00
Duration                    961 days 15:00:00
Exposure Time [%]                     51.0681
Equity Final [$]               10323859.69403
Equity Peak [$]                 11183060.7101
Return [%]                          932.38597
Buy & Hold Return [%]                41.80691
Return (Ann.) [%]                   142.47659
Volatility (Ann.) [%]               190.64259
CAGR [%]                            142.56036
Sharpe Ratio                          0.74735
Sortino Ratio                         3.77925
Calmar Ratio                           3.0544
Alpha [%]                           913.58314
Beta                                  0.44975
Max. Drawdown [%]                   -46.64639
Avg. Drawdown [%]                    -5.46467
Max. Drawdown Duration      286 days 08:00:00
Avg. Drawdown Duration        7 days 02:00:00
# Trades                                  511
Win Rate [%]                      

<Strategy SMAStrategy(sma_short_period=15,sma_long_period=25)>

# Now we will test the optimized parameters on the testing set
- Here, we pass in `df_test`, our testing dataset which contains unseen data.  
- We again call `.run()`, but this time we pass in our optimized parameters `.run(**stats_optimised._strategy._params)`. This runs the backtest using those parameters rather than the ones defined in the Strategy class.  
- Finally, we print the performance statistics of this backtest to see how our optimized parameters performed on the testing set.  

## Did we overfit?
- We want to determine whether the optimized parameters were overfitted to the training set.  
- Overfitting occurs when a strategy learns patterns that are too specific to the training data, failing to capture the general, repeatable signals we are aiming for.  
- As a result, it performs well on the training data but fails to replicate that performance on unseen testing data.
- We can compare performance on training set and testing set to determine if the parameters were overfit


In [9]:
bt_test = Backtest(df_test, SMAStrategy, cash=1000000, exclusive_orders=True, finalize_trades=True)
test_stats = bt_test.run(**stats_optimised._strategy._params)

#printing all stats side by side for comparison
print(f"{'Metric':<30} | {'TRAINING (Before)':<25} | {'TRAINING (Optimized)':<25} | {'TEST SET':<25}")
print("-" * 115)

for metric in stats.index:
    if metric not in ['_strategy', '_equity_curve', '_trades']:
        train_before_val = stats[metric]
        train_after_val = stats_optimised[metric]
        test_val = test_stats[metric]
        print(f"{metric:<30} | {str(train_before_val):<25} | {str(train_after_val):<25} | {str(test_val):<25}")


Metric                         | TRAINING (Before)         | TRAINING (Optimized)      | TEST SET                 
-------------------------------------------------------------------------------------------------------------------
Start                          | 2022-02-03 14:00:00       | 2022-02-03 14:00:00       | 2024-09-22 05:00:00      
End                            | 2024-09-22 05:00:00       | 2024-09-22 05:00:00       | 2025-11-05 19:00:00      
Duration                       | 961 days 15:00:00         | 961 days 15:00:00         | 409 days 14:00:00        
Exposure Time [%]              | 50.47955357921353         | 51.068096608248325        | 53.05665751195199        
Equity Final [$]               | 5292485.283929954         | 10323859.69403354         | 1177782.0335611496       
Equity Peak [$]                | 6026145.936968244         | 11183060.71010064         | 1876907.9647048493       
Return [%]                     | 429.2485283929954         | 932.385969403354  

# Plotting the optimised strategy on our test set to visualise its performance


In [10]:
bt_test.plot()


GridPlot(id='p1641', ...)